### A Practical Real-World Recommendation Engine (Hotels + Guides + Destinations)

The code below simulates:

                        • content-based similarity
                        • popularity
                        • freshness
                        • quality
                        • user-preference matching
                        • final scoring

### Step 1 — Import libraries

In [88]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import datetime


### Step 2 — Example dataset

You’ll replace this with your real data from your DB.

In [89]:
data = {
    'id': [1, 2, 3, 4],
    'name': ['Hotel A', 'Hotel B', 'Hotel C', 'Hotel D'],
    'tags': [
        'luxury pool spa resort',
        'budget family clean rooms',
        'mountain view hiking nature',
        'city center business hotel'
    ],
    'views': [540, 230, 1200, 320],
    'bookings': [120, 80, 300, 90],
    'saves': [75, 22, 210, 40],
    'rating': [4.6, 3.8, 4.9, 4.2],
    'date_added': [
        '2024-01-10', '2023-06-15', '2024-02-01', '2022-11-01'
    ]
}

df = pd.DataFrame(data)
df['date_added'] = pd.to_datetime(df['date_added'])


### Step 3 — Popularity score

In [90]:
def compute_popularity(row):
    return (row['views'] * 0.4) + (row['bookings'] * 1.0) + (row['saves'] * 0.6)

df['popularity_score'] = df.apply(compute_popularity, axis=1)

# Normalize 0–1
df['popularity_score'] = df['popularity_score'] / df['popularity_score'].max()
def compute_popularity(row):
    return (row['views'] * 0.4) + (row['bookings'] * 1.0) + (row['saves'] * 0.6)

df['popularity_score'] = df.apply(compute_popularity, axis=1)

# Normalize 0–1
df['popularity_score'] = df['popularity_score'] / df['popularity_score'].max()


### Step 4 — Freshness score

Newer hotels get a boost.

In [91]:
today = pd.Timestamp.today()

def compute_freshness(row):
    age_days = (today - row['date_added']).days
    return 1 / (1 + age_days)

df['freshness_score'] = df.apply(compute_freshness, axis=1)
df['freshness_score'] = df['freshness_score'] / df['freshness_score'].max()


### Step 5 — Quality score (based on rating)

In [92]:
df['quality_score'] = df['rating'] / 5.0   # Normalize to 0–1


### Step 6 — Content similarity (tags → vectors → cosine)

In [93]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(df['tags']).toarray()

similarity = cosine_similarity(vectors)


### Step 7 — User preference matching

In [94]:
user_interest = "mountain hiking nature"

user_vec = cv.transform([user_interest]).toarray()

user_similarity = cosine_similarity(user_vec, vectors)[0]


### Step 8 — Combine all scoring signals

In [95]:
df['similarity_score'] = user_similarity
df['similarity_score'] = df['similarity_score'] / df['similarity_score'].max()

df['final_score'] = (
    df['similarity_score'] * 0.6 +
    df['popularity_score'] * 0.2 +
    df['freshness_score'] * 0.1 +
    df['quality_score'] * 0.1
)


### Step 9 — Sort the recommendations

In [96]:
recommendations = df.sort_values(by='final_score', ascending=False)
print(recommendations[['name', 'final_score']])


      name  final_score
2  Hotel C     0.998000
0  Hotel A     0.272871
3  Hotel D     0.196435
1  Hotel B     0.190899


Step 10 — That’s your recommendation engine

It outputs the hotels ranked by:

       • how similar they are
       • how popular they are
       • how fresh they are
       • how high quality they are

This is exactly how mature real-world recommender systems behave.



How You Use This in Your Project

All you need is to replace the dummy df with:

      • your hotels
      • your guides
      • your destinations
      • your user history
      • your tags/keywords you already generated

Then plug this scoring step into your backend API: